# Hipóteses
Uma hipótese é uma suposição ou proposta que se faz com base em informações limitadas, com o objetivo de investigar uma possível relação entre variáveis ou fenômenos. No contexto de análise de dados, uma hipótese serve como ponto de partida para explorar como diferentes elementos de uma tabela podem estar conectados ao problema em questão, como a imprevisibilidade de renda. A hipótese estabelece uma relação esperada entre os dados, que pode ser testada e validada (ou refutada) através de métodos estatísticos e análise de dados. Isso permite entender melhor as dinâmicas que influenciam a variabilidade da renda e buscar soluções mais eficazes.

**Importação da base de dados através da biblioteca gdown**



**Utilizando o gdown para Conectar com a Base de Dados**

O `gdown` é uma ferramenta útil para fazer o download de arquivos diretamente do Google Drive utilizando a linha de comando. Ele facilita a transferência de arquivos hospedados no Google Drive para o ambiente local, permitindo que sejam utilizados em análises e projetos de dados.

Para conectar e baixar os arquivos da base de dados, utilizamos os seguintes passos:

1. **Instalação do gdown**: Primeiro, instalamos o `gdown` utilizando o comando `!pip install gdown`. Isso garante que a ferramenta esteja disponível no ambiente Jupyter Notebook.

2. **Definição dos IDs dos arquivos**: Cada arquivo no Google Drive possui um ID único. Definimos os IDs dos arquivos que queremos baixar (`file_id1` e `file_id2`).

3. **Construção das URLs de download**: Utilizamos os IDs dos arquivos para construir as URLs de download (`url` e `url2`).

4. **Download dos arquivos**: Utilizamos o `gdown.download()` para baixar os arquivos especificados pelas URLs. Os arquivos são salvos localmente com os nomes `base_01_agosto` e `base_dia_31_julho`.

Com esses passos, os arquivos são baixados e podem ser carregados em dataframes pandas para análise posterior.


In [ ]:
#instalação da biblioteca necessária
!pip install gdown

In [ ]:
# - baixando o gdown para conseguir fazer do download do arquivo

!pip install gdown

In [ ]:

import gdown

file_id1 = '1NcEwzYcpzceVJbSfnZ7qD3zHyqWp1v-5'
file_id2 = '1bxy-1s7VN6OT31esBt_IZdOzJVCYY0AZ'
url = f'https://drive.google.com/uc?id={file_id1}'
url2 = f'https://drive.google.com/uc?id={file_id2}'

# - https://docs.google.com/spreadsheets/d/1NcEwzYcpzceVJbSfnZ7qD3zHyqWp1v-5/edit?usp=sharing&ouid=114380254363966734810&rtpof=true&sd=true
# - https://docs.google.com/spreadsheets/d/1bxy-1s7VN6OT31esBt_IZdOzJVCYY0AZ/edit?usp=sharing&ouid=114380254363966734810&rtpof=true&sd=true

# Baixar o arquivo
gdown.download(url, 'base_01_agosto', quiet=False)
gdown.download(url2, 'base_dia_31_julho', quiet=False)


### Importando as bibliotecas

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Leitura das bases de dados que serão utilizadas

In [21]:
df1 = pd.read_excel("base_01_agosto")
df2 = pd.read_excel("base_dia_31_julho")

Modificações na tabela

In [6]:
#Drop das colunas que não estão presentes em ambos os DataFrames
df1 = df1.drop(columns=['IPCA ES', 'IPCA BR', '% Ating. Meta', 'Taxa Ac. TRI % PIB', 'CONCATENAR', 'Cliente', 'Origem', 'Desconto R$', 'Desc %', 'Vl Bruto', 'VL Tabela']) 
df2 = df2.drop(columns=['Cliente', 'Origem', 'Desconto R$', 'Desc %', 'Vl Bruto', 'VL Tabela'])

In [7]:
df2['day'] = 1 # Adição da coluna com informação dos dias
df2 = df2.rename(columns={"ano":"year","Mês":"month"}) # Renomeando para inglês as colunas
df2['Mês/ano'] = pd.to_datetime(df2[['year', 'month', 'day']]) # Formatação da data para o padrão datetime
df2 = df2.drop(columns=['year', 'month','day'])
df1 = df1.drop(columns={'Mês', 'Ano'}) # Drop das colunas já utilizadas

df1_df2 = pd.concat([df1,df2], ignore_index=True) # Concatenação dos DataFrames

df1_df2 = df1_df2.drop_duplicates() # Remoção das informações repetidas

df1_df2 = df1_df2.dropna(subset=['Setor']) # Remoção dos valores nulos da coluna "Setor"

df1_df2['VL Liquido Log'] = np.log10(df1_df2['Vl Liquido Final']) # Criação da coluna com os valores da tabela em log

df1_df2_date = df1_df2

df1_df2_date['year'] = df1_df2_date['Mês/ano'].dt.year # Transformação da data de uma coluna em data de colunas separadas
df1_df2_date['month'] = df1_df2_date['Mês/ano'].dt.month
df1_df2_date.set_index(['year','month'], inplace=True) # Colocação das datas como index da tabela
df1_df2_date.sort_index(level=[0, 1], inplace=True) # Adicionando as datas na ordem correta

df1_df2_date['VL Liquido Log'] = np.log10(df1_df2_date['Vl Liquido Final'])



In [8]:
df_2020 = df1_df2_date.loc[2020]
df_2021 = df1_df2_date.loc[2021]
df_2022 = df1_df2_date.loc[2022]
df_2023 = df1_df2_date.loc[2023]

## Hipótese 1: Número de contratos em relação aos meses

A quantidade de contratos assinados por mês tem uma correlação direta com a receita futura da Rede Gazeta, especialmente em meses de alta demanda publicitária, como períodos de eventos sazonais. Acredita-se que meses com maior volume de contratos, principalmente contratos de grande porte com clientes estratégicos, resultam em um aumento significativo da receita líquida da empresa. Além disso, meses com menor número de contratos não necessariamente indicam uma queda proporcional na receita, uma vez que a distribuição e a concentração de contratos em certos períodos podem criar fluxos de receita menos lineares, mas ainda relevantes para as projeções financeiras da empresa


In [ ]:
# Definindo a função plot_uen_by_year com o rótulo do eixo Y ajustado
def plot_uen_by_year(year, ax, y_max):
    df_year = df1_df2_date.loc[year]
    uen_count = df_year.groupby(['month', 'UEN']).size().unstack()
    uen_count.plot(kind='bar', ax=ax, title=f'Número de contratos por Mês e UEN em {year}')

    ax.grid(True, which='both', axis='both', linestyle='--', linewidth=0.7)

    # Definir limite do eixo Y
    ax.set_ylim(0, y_max)

    # Definir o rótulo do eixo Y
    ax.set_ylabel('número de contratos')

# Determinar o valor máximo do eixo Y com base no maior valor nos dados
y_max = df1_df2_date.groupby(['year', 'month', 'UEN']).size().max() + 50

# Criar um grid de subplots de 2x2
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

# Gerar gráficos para os anos de 2020 a 2023
plot_uen_by_year(2020, axs[0, 0], y_max)
plot_uen_by_year(2021, axs[0, 1], y_max)
plot_uen_by_year(2022, axs[1, 0], y_max)
plot_uen_by_year(2023, axs[1, 1], y_max)

# Mostrar os gráficos
plt.tight_layout()
plt.show()

## Hipótese 2: Valores de contratos em relação aos meses
O valor médio dos contratos assinados por mês tem uma influência direta e significativa na previsão da receita futura da Rede Gazeta. Meses com contratos de maior valor, impulsionados por grandes campanhas publicitárias ou acordos com clientes de alto perfil, tendem a elevar substancialmente a receita projetada para esses períodos. Em contrapartida, mesmo que o volume de contratos seja elevado, meses com um valor médio de contrato mais baixo podem resultar em uma receita futura inferior às expectativas. Ao combinar essa análise com a quantidade de contratos por mês, conforme explorado na primeira hipótese, é possível obter uma visão mais completa e robusta dos fatores que influenciam a receita futura da empresa, melhorando a precisão das projeções financeiras.

In [ ]:
# Definindo a função plot_mean_value_by_year para plotar os gráficos de média dos valores de contratos
def plot_mean_value_by_year(year, ax, y_max):
    df_year = df1_df2_date.loc[year]
    mean_values = df_year.groupby(['UEN', 'month'])['Vl Liquido Final'].mean().unstack()

    # Criar o gráfico de barras
    mean_values.plot(kind='bar', ax=ax, title=f'Valor Médio dos Contratos(R$) por UEN e Mês em {year}')

    # Adicionar grid
    ax.grid(True, which='both', axis='both', linestyle='--', linewidth=0.7)

    # Definir limite do eixo Y
    ax.set_ylim(0, y_max)

    # Definir o rótulo do eixo Y
    ax.set_ylabel('Valor Médio dos Contratos')

# Determinar o valor máximo do eixo Y com base no maior valor nos dados
y_max = df1_df2_date.groupby(['year', 'month', 'UEN'])['Vl Liquido Final'].mean().max() + 2500

# Criar um grid de subplots de 2x2
fig, axs = plt.subplots(2, 2, figsize=(15, 15))

# Gerar gráficos para os anos de 2020 a 2023
plot_mean_value_by_year(2020, axs[0, 0], y_max)
plot_mean_value_by_year(2021, axs[0, 1], y_max)
plot_mean_value_by_year(2022, axs[1, 0], y_max)
plot_mean_value_by_year(2023, axs[1, 1], y_max)

# Mostrar os gráficos
plt.tight_layout()
plt.show()

## Hipótese 3: Renda em relação a Unidade de négocios

A variação de receita entre as diferentes Unidades Estratégicas de Negócio (UEN) — TV, rádio e digital — exerce um impacto crucial na projeção das receitas futuras da Rede Gazeta. Cada UEN apresenta padrões específicos de variação ao longo dos anos, o que pode afetar significativamente a previsão global de receita. Por exemplo, a unidade de TV tende a gerar uma receita mais concentrada em determinados períodos do ano, como em grandes eventos ou campanhas publicitárias televisivas, enquanto a unidade de rádio pode apresentar uma receita mais constante, com picos em meses de maior audiência. A unidade digital, por sua vez, pode ser mais suscetível a rápidas mudanças de mercado, com uma variação de receita acelerada, impactada por tendências de consumo online ou novas oportunidades de publicidade digital.
Ao observar essas flutuações de receita entre as UENs, fica evidente que cada uma tem um papel distinto na composição da receita total da empresa. Essa análise é essencial não apenas para entender a distribuição de renda, mas também para aprimorar as previsões financeiras, permitindo ajustes mais precisos com base no desempenho específico de cada unidade. Complementando as hipóteses anteriores sobre o número e o valor dos contratos, essa abordagem oferece uma visão mais granular sobre como diferentes áreas da empresa contribuem para o resultado financeiro global.



In [ ]:
from matplotlib.ticker import FuncFormatter

# Função para formatar os valores do eixo Y em milhões
def millions(x, pos):
    return '%1.1fM' % (x * 1e-6)

# Valores médios de contratos por UEN para diferentes anos
mean_values_by_uen_2023 = df_2023.groupby('UEN')['Vl Liquido Final'].sum()
mean_values_by_uen_2022 = df_2022.groupby('UEN')['Vl Liquido Final'].sum()
mean_values_by_uen_2021 = df_2021.groupby('UEN')['Vl Liquido Final'].sum()
mean_values_by_uen_2020 = df_2020.groupby('UEN')['Vl Liquido Final'].sum()

# Criar o array de posições para cada grupo de barras
bar_width = 0.2
r1 = np.arange(len(mean_values_by_uen_2023))
r2 = [x + bar_width for x in r1]
r3 = [x + bar_width for x in r2]
r4 = [x + bar_width for x in r3]

# Plotar as barras lado a lado
plt.bar(r1, mean_values_by_uen_2023, color='skyblue', width=bar_width, edgecolor='grey', label='2023')
plt.bar(r2, mean_values_by_uen_2022, color='purple', width=bar_width, edgecolor='grey', label='2022')
plt.bar(r3, mean_values_by_uen_2021, color='#fe7f9c', width=bar_width, edgecolor='grey', label='2021')
plt.bar(r4, mean_values_by_uen_2020, color='red', width=bar_width, edgecolor='grey', label='2020')
# Adicionar título e labels
plt.title('Total de Valor dos Contratos por UEN em R$')
plt.xlabel('UEN')
plt.ylabel('Total de Valor dos Contratos')
plt.xticks([r + bar_width for r in range(len(mean_values_by_uen_2023))], mean_values_by_uen_2023.index)

# Adicionar grid e legenda
plt.grid(True, which='both', axis='y', linestyle='--', linewidth=0.7)
plt.legend()

# Definir o formatador de escala do eixo Y
plt.gca().yaxis.set_major_formatter(FuncFormatter(millions))

# Mostrar o gráfico
plt.show()

## Conclusão

A análise das hipóteses demonstra como diferentes aspectos dos contratos afetam a previsibilidade da renda de uma empresa. A variação no número e nos valores dos contratos ao longo dos meses, bem como a distribuição entre unidades de negócio, como televisão, rádio e digital, são fatores que podem tornar a receita imprevisível e dificultar o planejamento financeiro.

Para mitigar esses riscos, é essencial estabilizar o número de contratos, controlar as oscilações nos valores e diversificar as fontes de receita entre as unidades de negócio. Monitorar essas variáveis permite criar estratégias que garantam uma gestão financeira mais estável e um crescimento sustentável.